# Pandas

Pandas was developed by Wes McKinney while working as a researcher at AQR Capital performing quantitative analysis of financial data. Pandas introduced many features of the R programming language into Python.

- Pandas is the most used python package.
- Pandas has a very high and diverse industry adoption.
- **Pandas is huge in the data science and machine learning communities.**

## [Pandas Tutorial](https://pandas.pydata.org/docs/getting_started/index.html#intro-to-pandas)

In [ ]:
import numpy as np
import pandas as pd

## Pandas container for tabular data is called a `DataFrame`

### Pandas' usage model is to analyze/manipulate/plot data as a tablular object. 

#### Pandas supports a [huge number of filetypes](https://pandas.pydata.org/docs/user_guide/io.html#io) for reading and writing data


### We will use a dataset of Temps in Seattle over 2022 from [NOAA](https://www.ncdc.noaa.gov/cdo-web/search)

In [ ]:
# You can read a file via a URL

seatemp_table = pd.read_csv('https://uwashington-astro300.github.io/A300_Data/SeattleTemp_2022.csv',
                            parse_dates=['DATE'])

## Pieces


* `head()`
* `tail()`
* `loc[row, column]`

In [ ]:
seatemp_table.head(4)

In [ ]:
seatemp_table.tail(4)

### `pandas` slices are different than `numpy` slices - they include the end point

In [ ]:
seatemp_table.loc[2:3, ['TMAX', 'TMIN']]

### Notice that each row has an `index` assigned to it.

In [ ]:
seatemp_table.head(4)

In [ ]:
seatemp_table.info()

In [ ]:
seatemp_table.describe()

In [ ]:
seatemp_table.min()

##### `.min(), .max(), .mean(), .std()`

### Number of values

* Again, many ways to count

In [ ]:
seatemp_table.count()

In [ ]:
len(seatemp_table)

## Copy of a DataFrame

In [ ]:
other_table = seatemp_table.copy()

In [ ]:
other_table.head(5)

## Sorting (`.sort_values`)

In [ ]:
other_table.sort_values(['TMAX']).head(5)

In [ ]:
other_table.sort_values(
    ['TMAX'],
    ascending=False
).head(5)

#### The original table is unchanged

In [ ]:
other_table.head(5)

#### Add `inplace = True` to modify the original dataframe

In [ ]:
other_table.sort_values(
    ['TMAX'],
    inplace = True
)

#### The original table is changed

In [ ]:
other_table.head(5)

#### Notice that the row-index has **NOT** been reordered!

In [ ]:
other_table.loc[1:4, :]

## Resetting the index (`.reset_index`)

In [ ]:
other_table.reset_index(drop = True, 
                        inplace = True)

In [ ]:
other_table.head(5)

In [ ]:
other_table.loc[1:4, :]

## Picking out data (`.query`)

In [ ]:
seatemp_table.query("TMAX > 90")

## Adding a column - `insert`

`.insert(loc, column, value, allow_duplicates = False)`

In [ ]:
temp_variation = seatemp_table['TMAX'] - seatemp_table['TMIN']

temp_variation.head(5)

In [ ]:
# Add column in position 1 (2nd column)

seatemp_table.insert(1, 'T_RANGE', temp_variation, allow_duplicates = False)

In [ ]:
seatemp_table.head(5)

## Removing a column - `drop`

In [ ]:
seatemp_table.drop(columns='T_RANGE', inplace = True)

In [ ]:
seatemp_table.head(5)

## Adding a column (quick) - always to the end of the table

In [ ]:
seatemp_table['T_RANGE'] = temp_variation

In [ ]:
seatemp_table.head(5)

## Rearranging columns

In [ ]:
seatemp_table.columns

In [ ]:
my_new_order = ['DATE', 'T_RANGE', 'TMAX', 'TMIN']

In [ ]:
seatemp_table = seatemp_table[my_new_order]

In [ ]:
seatemp_table.head(5)

---

# Time

### Pandas really excels at handling time series. 

This is a direct result of pandas' financial-world origins. Although this functionality is partly based on NumPy time series functions, Pandas provides much more flexibility.


### In our example, the column `DATE` is a `datetime` object

- It has a ton of built-in functions that deal with time stuff

In [ ]:
seatemp_table.info()

### Find the number of days between the maximum `TMAX` and the minimum `TMIN`

In [ ]:
seatemp_table['TMAX'].idxmax()

In [ ]:
hot_date = seatemp_table['DATE'][seatemp_table['TMAX'].idxmax()]

hot_date

In [ ]:
cold_date = seatemp_table['DATE'][seatemp_table['TMIN'].idxmin()]

cold_date

In [ ]:
cold_date - hot_date

### What is the average `TMAX` for each month?

In [ ]:
seatemp_table['MONTH'] = seatemp_table['DATE'].dt.month

In [ ]:
seatemp_table.head(5)

In [ ]:
seatemp_table.groupby('MONTH')['TMAX'].mean()

---
## Time series functions

Pandas has a huge number of functions for working with time series data. One of the most common is the `rolling` function.

Here is an example of the `rolling` function to calculate the rolling mean.

- `NaN` = Not A Number

<img src="https://uwashington-astro300.github.io/A300_images/rolling_mean.jpg" width="400"/>

In [ ]:
seatemp_table['TMAX'].rolling(10).mean().head(20)

# Pandas ↔ QTable

In [ ]:
from astropy.table import QTable
from astropy import units as u
from astropy.units import imperial

In [ ]:
my_qtable_seatemp = QTable.from_pandas(seatemp_table)

In [ ]:
my_qtable_seatemp[0:5]

### You can then add and manipulate units

In [ ]:
my_qtable_seatemp['T_RANGE'].unit = imperial.deg_F
my_qtable_seatemp['TMAX'].unit = imperial.deg_F
my_qtable_seatemp['TMIN'].unit = imperial.deg_F

my_qtable_seatemp[0:5]

In [ ]:
my_qtable_seatemp['TMAX'].to(u.deg_C, equivalencies=u.temperature())

### ☹ Converting from QTables to DataFrames loses all unit information ☹

In [ ]:
my_pandas_seatemp = my_qtable_seatemp.to_pandas()

In [ ]:
my_pandas_seatemp.head(5)

---
# Pandas Plotting - `Seaborn`

* `Seaborn` is a library for making statistical graphics in Python
* `Seaborn` is tightly intergrated with the `pandas` library

## [Seaborn Tutorial](https://seaborn.pydata.org/tutorial.html)

In [ ]:
import seaborn as sns

In [ ]:
my_plot = sns.relplot(x = "DATE", y = "TMAX",
                      data=seatemp_table.head(31), 
                      kind = "line",
                      height = 8
                     )

### We can deal with the overlapping dates

In [ ]:
my_plot = sns.relplot(x = "DATE", y = "TMAX",
                      data = seatemp_table.head(31), 
                      kind = "line",
                      height = 8
                     )

my_plot.set_xticklabels(rotation = 65, 
                        horizontalalignment = 'right')

In [ ]:
my_plot = sns.relplot(x = "DATE", y = "TMAX",
                      data = seatemp_table.head(31), 
                      kind = "scatter",
                      height = 8
                     )

my_plot.set_xticklabels(rotation = 65, 
                        horizontalalignment = 'right')

In [ ]:
my_plot = sns.relplot(x = "DATE", y = "TMAX",
                      data = seatemp_table.head(31), 
                      kind = "scatter",
                      size = "T_RANGE",
                      height = 8
                     )

my_plot.set_xticklabels(rotation = 65, 
                        horizontalalignment = 'right')

## Seaborn really shines when doing statistical analysis of data

### `relplot()` =  relationship plot

In [ ]:
grade_table = pd.read_csv('https://uwashington-astro300.github.io/A300_Data/Grades.csv')

grade_table.head(3)

In [ ]:
sns.relplot(x = "Exam1", y = "Exam2",
            data = grade_table,
            height = 8);

In [ ]:
sns.relplot(x = "Exam1", y = "Exam2",
            data = grade_table,
            height = 8,
            hue = "Quarter");

In [ ]:
sns.relplot(x = "Exam1", y = "Exam2",
            data = grade_table,
            height = 8,
            hue = "Quarter",
            style = "Quarter");

In [ ]:
sns.jointplot(x = "Exam1", y = "Exam2",
            data = grade_table,
            height = 10,
            hue = "Quarter");

### `catplot()` =  category plot

In [ ]:
sns.catplot(x = "Quarter", y = "Exam2",
            data = grade_table,
            height = 8,
            kind='swarm');

In [ ]:
sns.catplot(x = "Quarter", y = "Exam2",
            data = grade_table,
            height = 8,
            kind='box');

In [ ]:
sns.catplot(x = "Quarter", y = "Exam2",
            data = grade_table,
            height = 8,
            kind='boxen');

In [ ]:
sns.catplot(x = "Quarter", y = "Exam2",
            data = grade_table,
            height = 8,
            kind='violin');